# Convolutional Neural Network

### Importing the libraries 


In [6]:
import tensorflow as tf 
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing (image augmentation)

### Preprocessing the training set
- rescale: _feature scalling_ because that's the representation range, this is a normalization 
- shear_range: is
- zoom_range = applies zoom over the images
- horizontal_flip = 


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255 #Feature scalling
    shear_range = 0.2
    zoom_range = 0.2
    horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary')


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary')

## Part 2 - Building the CNN


### Initialising the CNN


In [ ]:
cnn = tf.keras.models.Sequential()

### Setp 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size= 3, activation = 'relu',imput_shape=[64,64,3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))

### Adding a second convolutional layer


In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size= 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection


In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activations='relu'))

### Step 5 - Output Layer 

In [ ]:
cnn.add(tf.kears.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN


### Compiling the CNN


In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the training set an evaluation it on the test set


In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25,)

## Part 4 - Making a single prediction


In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog1.jpg',target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)